In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter

In [10]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [11]:
# Convolutional neural network (two convolutional layers)
class GRU_AT(nn.Module):
    def __init__(self, num_classes=4):
        super(GRU_AT, self).__init__()
           
        self.d1 = nn.Dropout(p=0.25)      
        self.bn1 = nn.BatchNorm1d(64)
        self.gru = nn.GRU(64,256,dropout=0.25)
        self.fc1 = nn.Linear(512, 64)
        self.fc2 = nn.Linear(64, num_classes)
        #attention parameter
        self.u = torch.randn(8,device=device,requires_grad=True)
        self.fc_attention = nn.Linear(512, 8)
        self.ac_attention = nn.Softmax(dim=1)
        
    def forward(self, x):    
       
        #b,64,64
        x, _= self.gru(x)
        #b,64,256
        temp1 = self.fc_attention(x)
        temp1 = F.tanh(temp1)
        #b,64,8
        alpha = torch.tensordot(temp1,self.u,1)
        alpha = self.ac_attention(alpha)
        #b,64 = b,64,8 8
        alpha = alpha.reshape(-1,64,1)
        #b,64,1
        x = torch.mul(x,alpha)
        #b,64,256 * b,64,1 = b,64,256
        x = torch.sum(x,dim=1)
        #b,64,256 -> b,,256
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.softplus(x)
        x = self.d1(x)
        x = self.fc2(x)

        return x

In [12]:
test = GRU_AT(4).to(device)
a = torch.randn(10,64,64)
a = a.to(device)
out = test(a)
out.shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (640x256 and 512x8)

In [ ]:
writer = SummaryWriter()

In [ ]:
train_set = pd.read_csv("data/final_format/train_set.csv",header=None).to_numpy()
train_label = pd.read_csv("data/final_format/train_label.csv",header=None).to_numpy()
test_set = pd.read_csv("data/final_format/test_set.csv",header=None).to_numpy()
test_label = pd.read_csv("data/final_format/test_label.csv",header=None).to_numpy()

In [ ]:
#delet first row data
train_set = train_set[1:]
train_label = train_label[1:]
test_set = test_set[1:]
test_label = test_label[1:]
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

In [ ]:
train_label = train_label.reshape(-1)
test_label = test_label.reshape(-1)
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

In [ ]:
#transpose to set data in time sequence
train_set = train_set.reshape(-1,64,64)
train_set = np.transpose(train_set,[0,2,1])
test_set = test_set.reshape(-1,64,64)
test_set = np.transpose(test_set,[0,2,1])
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

In [ ]:
# Hyper parameters
num_epochs = 300
num_classes = 4
batch_size = 64
learning_rate = 1e-3

In [ ]:
train_set_tensor = Tensor(train_set) 
train_label_tensor = Tensor(train_label).type(torch.LongTensor)

train_dataset = TensorDataset(train_set_tensor,train_label_tensor) 
train_loader = DataLoader(train_dataset, batch_size=batch_size) 

test_set_tensor = Tensor(test_set) 
test_label_tensor = Tensor(test_label).type(torch.LongTensor)

test_dataset = TensorDataset(test_set_tensor,test_label_tensor) 
test_loader = DataLoader(test_dataset, batch_size=batch_size) 

In [ ]:
model = GRU_AT(num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3) 
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
milestones = [50,100,150,200,250]
milestones = [a * len(train_loader) for a in milestones]
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.5)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    correct=0
    total=0
    running_loss = 0
    for i, (X, Y) in enumerate(train_loader):
        X = X.to(device)
        Y = Y.to(device)
        
        
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, Y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        #scheduler.step() 
        #print(scheduler.get_last_lr()[0])
      
        optimizer.step()
        scheduler.step() 
        #print(optimizer.param_groups[0]["lr"])
        
        _, predicted = outputs.max(1)
        total += Y.size(0)
        correct += predicted.eq(Y).sum().item()
        running_loss += loss.item()
        accu=100.*correct/total
        train_loss = running_loss/(i+1)
        print ('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.4f}%, Training Loss: {:.4f}%'.format(epoch+1, num_epochs, i+1, total_step, accu, train_loss))
    
   
        #writer.add_scalar(f'train/accuracy', accu, epoch)
        #writer.add_scalar(f'train/loss', train_loss, epoch)
        writer.add_scalars(f'train/accuracy_loss', {
            'accuracy': accu,
            'loss': train_loss,
        }, epoch)
        

In [ ]:
from playsound import playsound
playsound(u"sound.mp3")

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for X, Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
        total += Y.size(0)
        correct += (predicted == Y).sum().item()

    print('Test Accuracy : {} %'.format(100 * correct / total))

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')